# BentoML PyTorch MNIST Tutorial

Link to source code: https://github.com/bentoml/BentoML/tree/main/examples/pytorch_mnist/

Install required dependencies:

In [ ]:
%pip install -r requirements.txt

## Define the model

First let's define a simple PyTorch network

In [1]:
import torch
import torch.nn as nn


class SimpleConvNet(nn.Module):
    """
    Simple Convolutional Neural Network
    """

    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(26 * 26 * 10, 50),
            nn.ReLU(),
            nn.Linear(50, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
        )

    def forward(self, x):
        return self.layers(x)

    def predict(self, inp):
        """predict digit for input"""
        self.eval()
        with torch.no_grad():
            raw_output = self(inp)
            _, pred = torch.max(raw_output, 1)
            return pred

## Training and Saving the model

Then we define a simple PyTorch network and some helper functions

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import KFold

import bentoml

# reproducible setup for testing
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def _dataloader_init_fn(worker_id):
    np.random.seed(seed)

In [3]:
K_FOLDS = 5
NUM_EPOCHS = 5
LOSS_FUNCTION = nn.CrossEntropyLoss()


def get_dataset():
    # Prepare MNIST dataset by concatenating Train/Test part; we split later.
    train_set = MNIST(
        os.getcwd(), download=True, transform=transforms.ToTensor(), train=True
    )
    test_set = MNIST(
        os.getcwd(), download=True, transform=transforms.ToTensor(), train=False
    )
    return train_set, test_set


def train_epoch(model, optimizer, loss_function, train_loader, epoch, device="cpu"):
    # Mark training flag
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        if batch_idx % 499 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(inputs),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )


def test_model(model, test_loader, device="cpu"):
    correct, total = 0, 0
    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    return correct, total

In [4]:
# load data
train_set, test_set = get_dataset()
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=10,
    sampler=torch.utils.data.RandomSampler(test_set),
    worker_init_fn=_dataloader_init_fn,
)

### Cross Validation

We can do some cross validation and the results can be saved with the model as metadata


In [5]:
def cross_validate(dataset, epochs=NUM_EPOCHS, k_folds=K_FOLDS, device='cpu'):
    results = {}

    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=k_folds, shuffle=True)

    print("--------------------------------")

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

        print(f"FOLD {fold}")
        print("--------------------------------")

        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

        # Define data loaders for training and testing data in this fold
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=10,
            sampler=train_subsampler,
            worker_init_fn=_dataloader_init_fn,
        )
        test_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=10,
            sampler=test_subsampler,
            worker_init_fn=_dataloader_init_fn,
        )

        # Train this fold
        model = SimpleConvNet().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        loss_function = nn.CrossEntropyLoss()
        for epoch in range(epochs):
            train_epoch(model, optimizer, loss_function, train_loader, epoch, device)

        # Evaluation for this fold
        correct, total = test_model(model, test_loader, device)
        print("Accuracy for fold %d: %d %%" % (fold, 100.0 * correct / total))
        print("--------------------------------")
        results[fold] = 100.0 * (correct / total)

    # Print fold results
    print(f"K-FOLD CROSS VALIDATION RESULTS FOR {K_FOLDS} FOLDS")
    print("--------------------------------")
    sum = 0.0
    for key, value in results.items():
        print(f"Fold {key}: {value} %")
        sum += value

    print(f"Average: {sum/len(results.items())} %")

    return results

In [6]:
cv_results = cross_validate(train_set, epochs=1, device=device)

--------------------------------
FOLD 0
--------------------------------
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.273127
Train Epoch: 0 [4990/60000 (10%)]	Loss: 1.362216
Train Epoch: 0 [9980/60000 (21%)]	Loss: 0.411742
Train Epoch: 0 [14970/60000 (31%)]	Loss: 1.001366
Train Epoch: 0 [19960/60000 (42%)]	Loss: 0.102839
Train Epoch: 0 [24950/60000 (52%)]	Loss: 0.510927
Train Epoch: 0 [29940/60000 (62%)]	Loss: 0.053766
Train Epoch: 0 [34930/60000 (73%)]	Loss: 0.231184
Train Epoch: 0 [39920/60000 (83%)]	Loss: 0.372940
Train Epoch: 0 [44910/60000 (94%)]	Loss: 0.128234
Accuracy for fold 0: 91 %
--------------------------------
FOLD 1
--------------------------------
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.327491
Train Epoch: 0 [4990/60000 (10%)]	Loss: 1.318107
Train Epoch: 0 [9980/60000 (21%)]	Loss: 0.485991
Train Epoch: 0 [14970/60000 (31%)]	Loss: 1.216311
Train Epoch: 0 [19960/60000 (42%)]	Loss: 0.419982
Train Epoch: 0 [24950/60000 (52%)]	Loss: 0.758951
Train Epoch: 0 [29940/60000 (62%)]	Loss: 0

### training the model

In [7]:
def train(dataset, epochs=NUM_EPOCHS, device="cpu"):

    train_sampler = torch.utils.data.RandomSampler(dataset)
    train_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=10,
        sampler=train_sampler,
        worker_init_fn=_dataloader_init_fn,
    )
    model = SimpleConvNet()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    loss_function = nn.CrossEntropyLoss()
    model = model.to(device)
    for epoch in range(epochs):
        train_epoch(model, optimizer, loss_function, train_loader, epoch, device)
    return model

In [8]:
trained_model = train(train_set, device=device)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.371420
Train Epoch: 0 [4990/60000 (8%)]	Loss: 1.000866
Train Epoch: 0 [9980/60000 (17%)]	Loss: 1.110684
Train Epoch: 0 [14970/60000 (25%)]	Loss: 0.737674
Train Epoch: 0 [19960/60000 (33%)]	Loss: 0.364441
Train Epoch: 0 [24950/60000 (42%)]	Loss: 0.390488
Train Epoch: 0 [29940/60000 (50%)]	Loss: 0.282015
Train Epoch: 0 [34930/60000 (58%)]	Loss: 0.199639
Train Epoch: 0 [39920/60000 (67%)]	Loss: 0.367602
Train Epoch: 0 [44910/60000 (75%)]	Loss: 0.554657
Train Epoch: 0 [49900/60000 (83%)]	Loss: 0.017670
Train Epoch: 0 [54890/60000 (91%)]	Loss: 0.086635
Train Epoch: 0 [59880/60000 (100%)]	Loss: 0.249397
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.222882
Train Epoch: 1 [4990/60000 (8%)]	Loss: 0.287983
Train Epoch: 1 [9980/60000 (17%)]	Loss: 0.151059
Train Epoch: 1 [14970/60000 (25%)]	Loss: 0.118343
Train Epoch: 1 [19960/60000 (33%)]	Loss: 0.756619
Train Epoch: 1 [24950/60000 (42%)]	Loss: 0.100837
Train Epoch: 1 [29940/60000 (50%)]	Loss: 0.133997
Train Epoch: 1 

### saving the model with an input

In [9]:
import PIL.Image
import numpy as np

In [10]:
# save an image tensor for eager transpile
img = PIL.Image.open("samples/0.png")
np_img = np.array(img)
tensor_img = torch.from_numpy(np_img).float()
tensor_img = tensor_img.unsqueeze(0).unsqueeze(0)
tensor_img = torch.nn.functional.interpolate(tensor_img, size=28, mode='bicubic', align_corners=False).to(device)

In [11]:
correct, total = test_model(trained_model, test_loader, device=device)
metadata = {
    "accuracy": float(correct) / total,
    "cv_stats": cv_results,
}

tag = bentoml.pytorch.save_model(
    "pytorch_mnist",
    trained_model,
    metadata=metadata,
    custom_objects={'input': tensor_img}, # will allow eager transpile during load
)

# Loading model 

In [1]:
import torch
import PIL.Image
import numpy as np
import jax
import random

import bentoml

# reproducible setup for testing
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

jax.config.update("jax_enable_x64", True)

In [2]:
bentoml.models.list()

[Model(tag="pytorch_mnist:oac3zvakswac5nfv", path="/home/rishab/bentoml/models/pytorch_mnist/oac3zvakswac5nfv")]

### load flax version
after using ivy transpile magic

In [3]:
# device = "cuda:0"
device = 'cpu'

In [ ]:
flax_graph, params = bentoml.flax.load_model("pytorch_mnist:latest", ivy_transpile=True, device=device)

### load normal pytorch version

In [5]:
torch_model = bentoml.pytorch.load_model("pytorch_mnist:latest", device)

### compare outputs

In [6]:
img = PIL.Image.open("samples/0.png")
np_img = np.array(img)
tensor_img = torch.from_numpy(np_img).float()
tensor_img = tensor_img.unsqueeze(0).unsqueeze(0)
tensor_img = torch.nn.functional.interpolate(tensor_img, size=28, mode='bicubic', align_corners=False).to(device)

np_image = tensor_img.detach().cpu().numpy()

In [7]:
@jax.jit
def infer(np_image):
    return flax_graph.apply(params, np_image)

In [8]:
torch_model.to(device)

def _f(args):
  return torch_model(args)

comp_model = torch.compile(_f)

### inference latency comparision

In [9]:
%%timeit
_ = comp_model(tensor_img)

203 µs ± 115 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
_ = infer(np_image)

71.4 µs ± 9.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### sanity check
let’s load a different image and make sure that the results are the same in both models

In [12]:
img = PIL.Image.open("samples/3.png")
np_img = np.array(img)
tensor_img = torch.from_numpy(np_img).float()
tensor_img = tensor_img.unsqueeze(0).unsqueeze(0)
tensor_img = torch.nn.functional.interpolate(tensor_img, size=28, mode='bicubic', align_corners=False).to(device)
np_image = tensor_img.detach().cpu().numpy()

jax_outputs = infer(np_image)
torch_output = comp_model(tensor_img)

In [13]:
np.allclose(torch_output.detach().cpu().numpy(), jax_outputs, atol=1e-4)

True

In [14]:
torch.max(torch_output, 1)[1].item() == np.argmax(jax_outputs, axis=1)

Array([ True], dtype=bool)